In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

cwd = os.getcwd()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input 
apply_data_augmentation = False

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect',
                                      preprocessing_function=preprocess_input,
                                      validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect',
                                      validation_split=0.2)
else:
    train_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(validation_split=0.2)

# Create validation ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)
valid_mask_data_gen = ImageDataGenerator(validation_split=0.2)

# Create generators to read images from dataset directory
# -------------------------------------------------------

dataset_dir = '/content/drive/My Drive/AN2DL/Challenge2/Bip/Training256'
# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256
num_classes=3

# Training
# Two different generators for images and masks but with same seed
training_dir = os.path.join(dataset_dir, 'Mais') #here put 'Haricot' to train on haricot dataset
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'Images'),
                                                      target_size=(img_h, img_w),
                                                      batch_size=bs,
                                                      class_mode=None, 
                                                      shuffle=True,
                                                      interpolation='bilinear',
                                                      seed=SEED,
                                                      subset='training')  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'Masks'),
                                                        target_size=(img_h, img_w),
                                                        batch_size=bs,
                                                        class_mode=None,
                                                        shuffle=True,
                                                        interpolation='nearest',
                                                        seed=SEED,
                                                        subset='training')
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(training_dir, 'Images'),
                                                      target_size=(img_h, img_w),
                                                      batch_size=bs, 
                                                      class_mode=None, 
                                                      shuffle=True,
                                                      interpolation='bilinear',
                                                      seed=SEED,
                                                      subset='validation')
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'Masks'),
                                                        target_size=(img_h, img_w),
                                                        batch_size=bs, 
                                                        class_mode=None,
                                                        shuffle=True,
                                                        interpolation='nearest',
                                                        seed=SEED,
                                                        subset='validation')
valid_gen = zip(valid_img_gen, valid_mask_gen)

Found 3456 images belonging to 1 classes.
Found 3456 images belonging to 1 classes.
Found 864 images belonging to 1 classes.
Found 864 images belonging to 1 classes.


In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,img_h, img_w, 3], [None,img_h, img_w,3]))


def prepare_target(x_, y_):
  # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target labels 256 x 256 x 1
  #[254, 124, 18] = 0 done implicitly with the first cast
  #[255, 255, 255]= 1 done with the first cast
  #[216, 67, 82]= 2 done with the second cast that is multiplied by 2
    output=tf.cast(tf.reduce_all(y_== [255, 255, 255], axis=-1, keepdims=True), tf.float32)
    output=output+2*tf.cast(tf.reduce_all(y_== [216, 67, 82], axis=-1, keepdims=True), tf.float32)
    return x_, output

train_dataset = train_dataset.map(prepare_target)


train_dataset = train_dataset.repeat()



valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,img_h, img_w, 3], [None,img_h, img_w,3]))
valid_dataset = valid_dataset.map(prepare_target)


valid_dataset = valid_dataset.repeat()

In [ ]:
# Create Model using VGG16 as encoder
# ------------
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(None, None, 3))

finetuning = True

if finetuning:
    freeze_until = 17 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False


def create_model(depth, num_classes):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    model.add(vgg)
    
    start_f = 256
        
    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

In [ ]:
model = create_model(depth=5, 
                     num_classes=3)

# Visualize created model as a table
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, None, None, 512)   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 256)   1179904   
_________________________________________________________________
re_lu (ReLU)                 (None, None, None, 256)   0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 256)   0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 128)   295040    
_________________________________________________________________
re_lu_1 (ReLU)               (None, None, None, 128)   0

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#training with callbacks
callbacks = []

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)
# Learning rate adapter
# --------------
lr_adapter = True
if lr_adapter:
    lr_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, verbose=1, mode='auto', min_delta=0.0001, cooldown=0,restore_best_weights=True)
    callbacks.append(lr_adapter_callback)
    
#training
model.fit(x=train_dataset,
          epochs=50,
          steps_per_epoch=len(train_img_gen ),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen ), 
           callbacks=callbacks)

In [ ]:
#save model for final prediction
model.save('/content/drive/My Drive/AN2DL/Challenge2/Bip/SavedModel/Mais256_vgg.h5')

#model.save('/content/drive/My Drive/AN2DL/Challenge2/Bip/SavedModel/Haricot256_vgg.h5')

In [ ]:
#load the two model for predictions
#one model will predict all haricot datasets
#the other one is used for mais datasets

def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

model_haricot=tf.keras.models.load_model('/content/drive/My Drive/AN2DL/Challenge2/Bip/SavedModel/Haricot256_vgg.h5',custom_objects={'meanIoU':meanIoU})

model_mais=tf.keras.models.load_model('/content/drive/My Drive/AN2DL/Challenge2/Bip/SavedModel/Mais256_vgg.h5',custom_objects={'meanIoU':meanIoU})

In [ ]:
#prepare submission 

from PIL import Image
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

submission_dict = {}
groups=['Bipbip','Pead','Roseau','Weedelec']
hm=['Haricot','Mais']
for g in groups:
  for h in hm:
      final_path = '/content/drive/My Drive/AN2DL/Challenge2/Development_Dataset/Test_Dev/'+ str(g) + '/'+ str(h) +'/Images'
      image_filenames = next(os.walk(final_path))[2]   
      for img_name in image_filenames:
          img=Image.open(final_path+'/'+img_name).convert('RGB')
          img_width,img_heigth=img.size
          #vgg preprocessing
          img_array=np.array(img)
          img_array=preprocess_input(img_array)
          height = 256
          width = 256
          #creation of patches of subimages with size 256*256*3
          subimages=tf.image.extract_patches(images=tf.expand_dims(tf.convert_to_tensor(img_array), 0),
                           sizes=[1, width, height, 1],
                           strides=[1, width, height, 1],
                           rates=[1, 1, 1, 1],
                           padding='SAME') #pad with zeros in case of missmatching dimenzion
          

          number_subimages=subimages.shape[1]*subimages.shape[2]
          n_row=subimages.shape[1]
          n_col=subimages.shape[2]
          subimages=tf.reshape(subimages,[number_subimages,256,256,3])
          #we compute segmentation for each subimage and then we reconstrcut the final mask
          for i in range(0, n_row):
            for j in range(0,n_col):
              if h=='Haricot':
                  out_sigmoid = model_haricot.predict(x=tf.expand_dims(subimages[i*n_col+j,:,:,:], 0))
              else:
                  out_sigmoid = model_mais.predict(x=tf.expand_dims(subimages[i*n_col+j,:,:,:], 0))

              predicted_class = tf.argmax(out_sigmoid, -1)
              predicted_class = predicted_class[0, ...]
              mask_arr = np.array(predicted_class)
              if j==0:
                      mask_arr_row=mask_arr
              else:
                      mask_arr_row=np.hstack((mask_arr_row,mask_arr))
            if i==0:
                mask_arr_final=mask_arr_row
            else:
                mask_arr_final=np.vstack((mask_arr_final,mask_arr_row))

          mask_arr_final=mask_arr_final[:img_heigth,:img_width]   
          # RLE encoding
          # crop
          rle_encoded_crop = rle_encode(mask_arr_final == 1)
          # weed
          rle_encoded_weed = rle_encode(mask_arr_final == 2)
          img_name_without_extension=img_name[:-4]
          submission_dict[img_name_without_extension] = {}
          submission_dict[img_name_without_extension]['shape'] =mask_arr_final.shape
          submission_dict[img_name_without_extension]['team'] = g
          submission_dict[img_name_without_extension]['crop'] = h
          submission_dict[img_name_without_extension]['segmentation'] = {}
          submission_dict[img_name_without_extension]['segmentation']['crop'] = rle_encoded_crop
          submission_dict[img_name_without_extension]['segmentation']['weed'] = rle_encoded_weed
 


In [ ]:
#create json file for submission
import json
with open('/content/drive/My Drive/AN2DL/Challenge2submission.json', 'w') as f:
  json.dump(submission_dict, f)